# Encontrando números primos

Encontrar numeros primos siempre ha representado un reto, pues aunque su definición parezca trivial, entre más grande sea un número $n$ mayor es el número de interaciones necesarias para poder comprobar si este número es o no un número primo. Primero recordamos que un número primo es un numero $n\in \mathbb{N} $ tál que $ \forall\  a \in \mathbb{N},\ a\in (1,n)$, $n/a \not\in \mathbb{N}$

Una forma trivial de atacar este problema es por inspección, para esto hacemos una función que pruebe pueda comprobar si un numero dado $n$ es primo o no.

In [1]:
import numpy as np
import math
import time
from collections import Counter

In [2]:
def basic_is_prime(n):
    result=True
    for i in range(2,n):
        if n%i==0:
            result = False
    return result

Esta función recibe un numero entero $n$ y revisa todos los números hasta $n-1$ buscando los posibles divisores de $n$ en el caso en el que ningún número divida a $n$ entre $2$ y $n-1$, se concluye entonces que $n$ es primo y por ende la función `basic_is_prime`retorna `True`.
Ahora vamos a usar esta función para encontrar los primeros números primos entre $0$ y $50000$.

In [3]:
start=time.time()
lista_basic_prime=[]
for i in range(2,50000):
    if basic_is_prime(i):
        lista_basic_prime.append(i)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 104.28834700584412


Como se ve, esto es un algoritmo lento, si se quisiera determinar si un número grande es o no un número primo esto podría tardar demasiado, con lo cual nos lleva a considerar un algoritmo distinto para poder determinar si algún numero es primo o no.

## Algoritmo mejorado

suponga que $m=\sqrt{n}$, entonces $m\times m = n$. Ahora si $n$ no es un número primo entonces $n$ puede ser escrito como $a\times b \Rightarrow \ m\times m=a\times b$. Observemos que si $m$ es un numero real, mientras que n,a y b son numeros naturales.

Ahora tenemos 3 posibles casos:
1. $a>m \Rightarrow b<m$
2. $a=m \Rightarrow b=m$
3. $a<m \Rightarrow b>m$

Para todos estos 3 casos, se tiene que $\min (a,b)\leq m$. Por ende $m$ será una cota para encontrar al menos un factor de $n$, lo cual resulta ser una condición suficiente para mostrar que $n$ no es primo.

In [4]:
def new_is_prime(n):
    result=True
    for i in range(2,int(np.sqrt(n)+1)):
        if n%i==0:
            result = False
    return result

In [5]:
start=time.time()
lista_new_prime=[]
for i in range(2,50000):
    if new_is_prime(i):
        lista_new_prime.append(i)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 0.7197661399841309


Como se puede ver esto es un mejora significativa en el programa, sin embargo esto podría ser mejorado ligeramente más. Para esto se hace uso del hecho de que ningun número par puede ser un número primo, con lo cual la busqueda de puede hacerce de en los impares.

In [6]:
def list_of_primes(n):
    lista=np.array([2])
    for i in range(3,n+1,2):
        if new_is_prime(i):
            lista=np.append(lista,i)
    return lista

In [7]:
start=time.time()
lista_new_prime_2=list_of_primes(50000)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 0.4051802158355713


In [8]:
len(lista_basic_prime),len(lista_new_prime),len(lista_new_prime_2)

(5133, 5133, 5133)

Una de las aplicaciones que tiene el encontrar los numeros primos es la descomposición de un número en factores primos, una forma de implementar esto se podría hacer de la siguiente forma: 

# ------ Sugerir como ejercicio ------

Dado $n\in \mathbb{N}$ encuentre la descomposición en factores primos de $n$. el resultado debe estar dado como una cadena escrita de la forma:

$"(p1**n1)(p2**n2)...(pk**nk)"$

con $p(i)$ está ordenado de menor a mayor y $n(i)$ es vacio si $n(i)$ es 1.

Ejemplo: n= 86240, la función retorna $"(2** 5)(5)(7** 2)(11)"$

In [9]:
def get_prime_Factors(n):
    if type(n) is not int or n < 1:
        # Not an integer, negative or 0
        return []
    elif(n<3):
        return [n]
    factors=[]
    if new_is_prime(n):
        return [n]
    while(n>1):
        for i in range(2,n+1):
            if new_is_prime(i) and not n % i:
                n=int(n/i)
                factors.append(i)
                break
    return factors

In [10]:
def primeFactors(n):
    f= get_prime_Factors(n)
    factors = sorted(list(set(f)))
    powers = [f.count(factor) for factor in factors]
    result=""
    for i in range(len(factors)):
        if powers[i]==1:
            result+="("+str(factors[i])+")"
        else:
            result+="("+str(factors[i])+"**"+str(powers[i])+")"
    return result

Usemos la anterior función para calcular la descomposición en numeros primos de $512345021, 777546031$ y $7775460$

In [11]:
print(primeFactors(512345021),primeFactors(777546031),primeFactors(7775460))

(512345021) (23567)(32993) (2**2)(3**3)(5)(7)(11**2)(17)


[Propuesta de la Solucion a este ejercicio](https://github.com/d2718nis/codewars-prime-number-decompositions)

# Metodos aleatorios para encontrar números primos

## Test de Fermat
Este es un test de complejidad $\mathbb{O}(\log(n)$ para comprobar si un número es primo o no. Este test está basado en _Fermat's Little Theorem_

**Fermat's Little Theorem**: si $n$ es un numero primo y $a$ es cualquier entero positivo menor que $n$, entonces $a^{n}$ es congruente a $a$ modulo $n$

Entonces si $n$ no es un numero primo, entonces en general para casi cualquier $a<n$ no se satisface el aterior teorema.

Este test difiere de la mayoria de los algoritmos convencionales, en la cual la respuesta del algoritmo garantiza que esta solución está correcta. Acá lo único que se puede decir de la respuesta proporcionada es que es probablemente correcta. De forma más precisa, si $n$ nunca falla el test de Fermat, se puede estar seguro de que $n$ es un número primo. Sin embargo, el hecho de que $n$ pase el test o no, no garantiza que $n$ sea primo o no, sólo se tiene un indicador de que muy probablemente lo sea. En resumen si este test se aplica una cantidad suficiente de veces y se tiene que $n$ siempre pasa el test, entonces la probabilidad de error al momento de decidir si este número es primo, se reduce tanto como se desee. 


El problema de este test aparece cuando se tiene que hacer la operación $a^{n}\ ({\mbox{mod}}\ n)$, pues si se hace de una forma "naïve" se tiene que la potencia $a^{n}$ crece muy rapido y esto hace que la la maquina no lo pueda escribir.
Para esto es entonce necesario hacer uso de otro tipo exponenciación, llamado "Exponenciación modular", este tipo de exponenciación resulta ser bastante útil en criptografia y esta aplicación resulta importante al tratar con números "Grandes".

[Link de referencias](https://en.wikipedia.org/wiki/Modular_exponentiation)

[link para hacer el algoritmo de exponenciación](https://eli.thegreenplace.net/2009/03/28/efficient-modular-exponentiation-algorithms)

In [12]:
def expmod(base, exp, n):
    r = 1
    for i in range(exp):
        r = r * base % n
    return r
#------- al parecer no funcionan -----
def modexp_rl(a, b, n):
    r = 1
    while 1:
        if b % 2 == 1:
            r = r * a % n
        b /= 2
        if b == 0:
            break
        a = a * a % n
    return r
def modexp_lr(a, b, n):
    r = 1
    for bit in reversed(_bits_of_n(b)):
        r = r * r % n
        if bit == 1:
            r = r * a % n
    return r
def _bits_of_n(n):
    """ Return the list of the bits in the binary
        representation of n, from LSB to MSB
    """
    bits = []

    while n:
        bits.append(n % 2)
        n /= 2

    return bits

Para implementar el Test de Fermat primero se se escoge un numero aleatorio $a$ entre $1$ y $n-1$ y se verifica si el residuo de modulo $n$ de $a^{n}$ es igual a $a$.

In [13]:
def Fermat_Test(n,k):
    Test=True
    #a=np.random.choice(range(2,n),k,replace=False)
    #exp=expmod(a,n,n)
    #for i in range(len(a)):
     #   if a[i] != exp[i]:
      #      Test=False
       #     break
    for i in range(k):
        a=np.random.randint(2,n)
        exp=expmod(a,n,n)
        if exp != a:
            Test=False
            break
    return Test

In [14]:
start=time.time()
nueva_lista=[2]
for i in range(3,50000):
    if Fermat_Test(i,10):
        nueva_lista.append(i)
end=time.time()
print("Tiempo tomado:",end-start)

Tiempo tomado: 221.70036578178406


Lo que se tiene acá es que aparentemente se está tomando más tiempo con este algoritmo que con el el propuesto anteriormente, no obstante, esto no es debido a que la cantidad de numero que está calculando no es exactamente la misma cantidad de números primos que hallamos anteriormente, esto se puede ver preguntando por la longitud de cada lista.

In [15]:
len(nueva_lista),len(list_of_primes(50000))

(5145, 5133)

Para ver cuales fueron los elementos que se agregaron de más, podemos comparar las listas.

In [16]:
lista_primos=list_of_primes(50000)
CH_num=[]
for i in range(len(nueva_lista)):
    if nueva_lista[i] not in lista_primos:
        CH_num.append(nueva_lista[i])
        #print(nueva_lista[i])
print(CH_num)

[561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 41041, 46657]


Se tiene entonces que los números que aparecieron son $561,1105,1729,2465,2821,6601,8911,10585, 15841, 29341, 41041, 46657 \ldots$, estos números como podemos ver no son primos pues tienen una descomposición en primos dada por:

In [17]:
for i in CH_num:
    print(i, "Se descompone como",primeFactors(i))

561 Se descompone como (3)(11)(17)
1105 Se descompone como (5)(13)(17)
1729 Se descompone como (7)(13)(19)
2465 Se descompone como (5)(17)(29)
2821 Se descompone como (7)(13)(31)
6601 Se descompone como (7)(23)(41)
8911 Se descompone como (7)(19)(67)
10585 Se descompone como (5)(29)(73)
15841 Se descompone como (7)(31)(73)
29341 Se descompone como (13)(37)(61)
41041 Se descompone como (7)(11)(13)(41)
46657 Se descompone como (13)(37)(97)


Esto no es conincidencia ni es problema de la cantidad de test que se efectuen, estos números son unos números bastante particulares llamado números de Carmichael y surgen del hecho de que el teorema de Fermat establece que si $a$ no es divisible por $p$ entonces $p$ divide a $a^{p}-a$:

$${\displaystyle \ p|a^{p}-a}.$$

Para determinar si un número n es primo o no, se escoge un número a que sea primo relativo con n y se calcula ${\displaystyle \ a^{n-1}{\pmod {n}}}$. Si el resultado es diferente a 1, el número es compuesto con toda certeza.

Desafortunadamente si el resultado es 1 no es posible asegurar que el número n es primo, ya que el inverso del teorema de Fermat no es válido: existen números compuestos a tales que ${\displaystyle a^{n}\equiv a{\pmod {n}}}$. Tales números se denominan pseudoprimos en la base a, por lo que la prueba propuesta no es en realidad una verdadera prueba de primalidad.

Los números de Carmichael son entonces números pseudoprimos en cualquier base: son los números para los que la prueba anterior falla para cualquier elección de base que sea primo relativo con el número dado.

## Ejercicio
Obtenga los primeros 20 números de Carmichael, para esto deberá hacer uso de algo más eficiente que el algoritmo planteado anteriormente.

$561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 41041, 46657, 52633, 62745, 63973, 75361, 101101, 115921, 126217, 162401$

# Algoritmo de Pollard's Rho

In [34]:
def g(x,n):
    return (x**2 +1)%n

In [ ]:
n=777546031
x=np.random.randint(2,n)
y=np.random.randint(2,n)
d=1
while(d==1):
    #print(x,y)
    x= g(x,n)
    y=g(g(y,n),n)
    d=math.gcd(np.abs(x-y),n)
print(d)

6984

[link para hacer el algoritmo de exponenciación](https://eli.thegreenplace.net/2009/03/28/efficient-modular-exponentiation-algorithms)

## Solución descomposición primera version

In [15]:
lista= list_of_primes(100000)

In [16]:
def primeFactors_first(n):
    resto=n
    continuar=True
    val=[]
    while(continuar):
        for i in lista:
            if resto%i==0:
                resto=resto/i
                val.append(i)
                break        
        if resto==1:
            continuar=False
    dicc=Counter(val)
    result=""
    for i in sorted(dicc):
        if dicc[i]==1:
            result+="("+str(i)+")"
        else:
            result+="("+str(i)+"**"+str(dicc[i])+")"
    return result